In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


#!/usr/bin/env python3

import pandas as pd

from tqdm import tqdm
tqdm.pandas()
pd.options.display.max_columns = None
pd.options.display.max_rows = None

import gzip
import json
import itertools
import sys
import numpy as np
import pickle
import time

import csv
import os
import collections
from collections import Counter
import networkx as nx

In [ ]:

exp_id = int(sys.argv[1])


df = pd.read_pickle('../Data/Final_retweet_dataframe.pkl')
print('len df',len(df))
def drop_blank(df_org):
    blank = ''
    df_drop = df_org[df_org['org_id'] != blank]
    df_drop = df_drop[df_drop['retweet_id'] != blank]
    df_drop = df_drop[df_drop['retweet_user'] != blank]
    df_drop = df_drop[df_drop['org_user'] != blank]
    
    return df_drop
    
df_drop = drop_blank(df)  
print('len df_drop',len(df_drop))


nodes = pd.read_pickle("/mnt/irisgpfs/users/nchen/COVID19/Multilingual/Data/wcc_nodes_link.list")
link_org = pd.read_pickle("/mnt/irisgpfs/users/nchen/COVID19/Multilingual/Data/wcc_edges_link.list")

link = [list(i) for i in link_org]

In [ ]:
cas_df = df_drop[df_drop['retweet_user'].isin(nodes)]

def print_info_df(df_final_in):
    """
    print information
    """
    df_final_in_user = df_final_in.drop_duplicates(subset=['retweet_user'], keep='last')
    print('number of retweet users',len(df_final_in_user)) 
    df_final_in_org_user = df_final_in.drop_duplicates(subset=['org_user'], keep='last')
    print('number of orignal users',len(df_final_in_org_user))
    df_final_in_tweet = df_final_in.drop_duplicates(subset=['org_id'], keep='last')
    print('number of orignal tweets',len(df_final_in_tweet))
    print('number of tweets',len(df_final_in))
    



In [ ]:

cas = cas_df.drop_duplicates(subset = ['retweet_id'], keep='last')

In [ ]:
print_info_df(cas)



print(cas.info())
org_id_l = cas['org_id'].tolist()
org_id = sorted(set(org_id_l), key=org_id_l.index)

In [ ]:

def select_cascade(stuff,link):
    cas_list = []
    len_inter = [itertools.combinations(stuff, 2)]
    for i in itertools.combinations(stuff, 2):
        a = [i[1], i[0]]
        if a in link:
            if len(cas_list) > 0:
                cas_l = []  # the cascade of a
                waiting = []  # check if duplicate
                remo = []  # remove orginal link from cas
                tar = []
                for m in cas_list:
                    tar.append(m[len(m) - 1])
                sor = i[0]
                if any(ele == sor for ele in tar):
                    for indx in range(len(tar)):
                        ex_l = cas_list[indx]
                        if tar[indx] == sor:
                            cas_l.append(ex_l + [i[1]])
                            remo.append(ex_l)

                    cas_list = cas_list + cas_l
                    # for r in remo:
                    #     cas.remove(r)
                else:
                    cas_list.append([i[0], i[1]])

            else:
                cas_list.append([i[0], i[1]])  # in link but not exit in cas_list
        else:
            if [i[0]] not in cas_list and len(len_inter) == 1:
                cas_list.append([i[0]])
                cas_list.append([i[1]])
            if [i[0]] not in cas_list and len(len_inter) != 1:
                cas_list.append([i[0]])
    return cas_list


if exp_id==0:
    idx1 = 0
    idx2 = int(len(org_id)/10 * (exp_id+1))
else:
    idx1 = int(len(org_id)/10 * (exp_id))
    idx2 = int(len(org_id)/10 * (exp_id+1))

print(f'from {idx1} to {idx2}')


In [ ]:


cascade = []
#for i in tqdm(org_id[:10]):
for i in tqdm(org_id[idx1:idx2]):
    a = cas.loc[(i,slice(None)),:]
    if len(a) >1:
        re = a['retweet_user'].tolist()
        re_user = []
        for i in re:
            if i not in re_user:
                re_user.append(str(i))
        # re_user = list(set(re_user))
        # re_user = [str(i) for i in re_user]
        or_user = str(a['org_user'].tolist()[0])

        cas_final = []
        cas_list = select_cascade(re_user,link)

        if len(cas_list) == 0:
            for x in re_user:
                cas_final.append([or_user,x])
        else:
            for x in cas_list:
                c = [or_user] + x
                cas_final.append(c)
                #print(cas_final)
        cascade.append(cas_final)

    if len(a) == 1:
        re_user = str(a['retweet_user'].tolist()[0])
        or_user = str(a['org_user'].tolist()[0])
        c = [or_user,re_user]
        cascade.append([c])

 #
print(cascade[0])
 # # # # #
with open('../Data/Cascade/select_out_cascade'+str(exp_id), 'wb') as f:
    pickle.dump(cascade, f)

In [ ]:
c = cascade


print(len(c))

cascade = []


new_cascade = []
mul = []
for l in tqdm(c):
    tweet_l = []
    nun_l = []
    c_ = []
    for element in l:
        for i in l:
            for element in l:
                check = all(item in element for item in i)
                if check is True and element != i:
                    tweet_l.append(i)

    for t in tweet_l:
        if t not in nun_l:
            nun_l.append(t)

    for m in l:
        if m not in nun_l:
            c_.append(m)
    new_cascade.append(c_)

print(len(new_cascade))

#
cas_df = cas[idx1:idx2]

with open('../Data/Cascade/clean_cascade'+str(exp_id), 'wb') as f:
    pickle.dump(new_cascade, f)

cascade_dict = {}
for i in range(len(new_cascade)):
    cascade_dict[org_id[i]] = new_cascade[i]

with open('../Data/Cascade/cascade_org_id'+str(exp_id), 'wb') as f:
    pickle.dump(org_id, f)
    
with open('../Data/Cascade/cascade_dict'+str(exp_id), 'wb') as f:
    pickle.dump(cascade_dict, f)
    
    
    